In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [6]:
# Import netcdf-file
filename = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2023/10/10/meps_lagged_6_h_subset_2_5km_20231010T00Z.nc'
data = xr.open_dataset(filename)

airport_file= pd.read_csv('data/airports_overview.csv')
airport_identifiers = airport_file['airport_identifier']

In [7]:
# Function for calculating the nearest latitude and longitude coordinates based on pre-defined target values
# 

def find_nearest_coordinates(airport_identifiers, target_lat, target_lon):
    latitude_values = data['latitude'].values
    longitude_values = data['longitude'].values

    # Calculate the euclidean distance from the target coordinates
    distances = np.sqrt((latitude_values - target_lat)**2 + (longitude_values - target_lon)**2)

    # Find indices of the minimum distance
    nearest_index = np.unravel_index(distances.argmin(), distances.shape)

    # Extract corresponding y and x indices
    nearest_y_index = nearest_index[0]
    nearest_x_index = nearest_index[1]

    # Extract data using found indices
    print(f'For airport {airport_identifiers}: Successfully extracted (y,x) for target_latitude = {target_lat} and target_longitude: {target_lon}')
    return nearest_y_index, nearest_x_index


In [15]:
# Function for locating the value of the corresponding x- and y-values based on index found from find_nearest_coordinates

def extract_vals_to_df(data_param, y_index, x_index):
    
    data_param = data_param.sel(   
        y = data['y'][y_index],   
        x = data['x'][x_index],
        method='nearest'
     )
    data_param = data_param.to_dataframe()
    return data_param

### Dataset created from extracted positions from postgresql database

In [9]:
airport_location = pd.read_csv('data/airport_positions.csv')
airport_location

,airport_identifier,name,position
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)"
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)"
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)"
3,ENAT,ALTA RWY 11/29,"(6997,2337)"
4,ENBJ,BJORNOYA,"(7450,1908)"
...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)"
86,ENWV,VALHALL A,"(5627,339)"
87,ENVR,VAEROY,"(6765,1272)"
88,ENXW,GRANE,"(5916,248)"


In [10]:
# Remove parenthesis -> split lon and lat values -> divide by 100 to obtain correct values
airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)
airport_location[['latitude', 'longitude']] = airport_location[['latitude', 'longitude']].astype('float') / 100

/tmp/ipykernel_464076/1650021312.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)


In [11]:
airport_location

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)",78.92,11.87
3,ENAT,ALTA RWY 11/29,"(6997,2337)",69.97,23.37
4,ENBJ,BJORNOYA,"(7450,1908)",74.50,19.08
...,...,...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)",70.06,29.84
86,ENWV,VALHALL A,"(5627,339)",56.27,3.39
87,ENVR,VAEROY,"(6765,1272)",67.65,12.72
88,ENXW,GRANE,"(5916,248)",59.16,2.48


In [16]:
# Function for processing each row from each airport through earlier defined functions

def process_airport_row(row):
    target_lat = row['latitude']
    target_lon = row['longitude']
    airport = row['airport_identifier']

    nearest_y_index, nearest_x_index = find_nearest_coordinates(airport, target_lat, target_lon)
    
    result_df = extract_vals_to_df(data['air_temperature_2m'], nearest_y_index, nearest_x_index)
    
    return result_df

In [17]:
test_dataset = airport_location.head(2)
test_dataset

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14


In [18]:
# Applying to dataframe
result = test_dataset.apply(process_airport_row, axis=1)

# Concatenate the results into a final DF
final_result = pd.concat(result.tolist(), keys=test_dataset['airport_identifier'])

final_result

For airport ENAL: Successfully extracted (y,x) for target_latitude = 62.56 and target_longitude: 6.11
For airport ENAN: Successfully extracted (y,x) for target_latitude = 69.29 and target_longitude: 16.14


x  \
airport_identifier time                height1 ensemble_member                  
ENAL               2023-10-10 00:00:00 2.0     0               -455084.062500   
                                               1               -455084.062500   
                                               2               -455084.062500   
                                               3               -455084.062500   
                                               4               -455084.062500   
...                                                                       ...   
ENAN               2023-10-12 13:00:00 2.0     25                44915.945312   
                                               26                44915.945312   
                                               27                44915.945312   
                                               28                44915.945312   
                                               29                44915.945312   

                                                                           y  \
airport_identifier time                height1 ensemble_member                 
ENAL               2023-10-10 00:00:00 2.0     0                -50017.90625   
                                               1                -50017.90625   
                                               2                -50017.90625   
                                               3                -50017.90625   
                                               4                -50017.90625   
...                                                                      ...   
ENAN               2023-10-12 13:00:00 2.0     25               667482.12500   
                                               26               667482.12500   
                                               27               667482.12500   
                                               28               667482.12500   
                                               29               667482.12500   

                                                                longitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                 6.089178   
                                               1                 6.089178   
                                               2                 6.089178   
                                               3                 6.089178   
                                               4                 6.089178   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               16.135429   
                                               26               16.135429   
                                               27               16.135429   
                                               28               16.135429   
                                               29               16.135429   

                                                                 latitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                62.565422   
                                               1                62.565422   
                                               2                62.565422   
                                               3                62.565422   
                                               4                62.565422   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               69.287653   
                                               26               69.287653   
                                               27               69.287653   
                                               28               69.287653   
                                               29      

In [87]:
final_result.head(50)

x  \
airport_identifier time                height1 ensemble_member                
ENAL               2023-10-10 00:00:00 2.0     0               -455084.0625   
                                               1               -455084.0625   
                                               2               -455084.0625   
                                               3               -455084.0625   
                                               4               -455084.0625   
                                               5               -455084.0625   
                                               6               -455084.0625   
                                               7               -455084.0625   
                                               8               -455084.0625   
                                               9               -455084.0625   
                                               10              -455084.0625   
                                               11              -455084.0625   
                                               12              -455084.0625   
                                               13              -455084.0625   
                                               14              -455084.0625   
                                               15              -455084.0625   
                                               16              -455084.0625   
                                               17              -455084.0625   
                                               18              -455084.0625   
                                               19              -455084.0625   
                                               20              -455084.0625   
                                               21              -455084.0625   
                                               22              -455084.0625   
                                               23              -455084.0625   
                                               24              -455084.0625   
                                               25              -455084.0625   
                                               26              -455084.0625   
                                               27              -455084.0625   
                                               28              -455084.0625   
                                               29              -455084.0625   
                   2023-10-10 01:00:00 2.0     0               -455084.0625   
                                               1               -455084.0625   
                                               2               -455084.0625   
                                               3               -455084.0625   
                                               4               -455084.0625   
                                               5               -455084.0625   
                                               6               -455084.0625   
                                               7               -455084.0625   
                                               8               -455084.0625   
                                               9               -455084.0625   
                                               10              -455084.0625   
                                               11              -455084.0625   
                                               12              -455084.0625   
                                               13              -455084.0625   
                                               14              -455084.0625   
                                               15              -455084.0625   
                                               16              -455084.0625   
                                               17              -455084.0625   
                                               18              -455084.0625   
                                             

### METAR

In [104]:
metar_data = pd.read_csv('data/metar_msg_overview.csv')

In [105]:
# Sort data based on airport and timestamp
metar_data.sort_values(['airport_identifier', 'issued_at'], inplace=True)

# Convert issued_at to datetime
metar_data['issued_at'] = pd.to_datetime(metar_data['issued_at'])

# Set airport and issued_at as index
metar_data.set_index(['airport_identifier', 'issued_at'], inplace=True)

# Group by airport
metar_data.groupby(['airport_identifier'])

In [106]:
# Replace NaN with MANUAL
metar_data['metartype'].fillna('MANUAL', inplace=True)

In [108]:
# Adjusting width of column to display the whole metar message
pd.set_option('display.max_colwidth', None)

In [98]:
metar_data.loc['ENGM', :]

,metar,metartype
issued_at,,
2021-01-01 00:20:00,ENGM 010020Z 36006KT 9999 4900E -SN FEW009 OVC017 M02/M03 Q1003=,MANUAL
2021-01-01 00:50:00,ENGM 010050Z 36003KT 9000 -SN FEW007 OVC018 M02/M03 Q1003=,MANUAL
2021-01-01 01:20:00,ENGM 010120Z 36006KT 8000 -SN FEW008 OVC029 M02/M03 Q1003=,MANUAL
2021-01-01 01:50:00,ENGM 010150Z 36006KT 9999 6000 R19R/P2000N R01R/P2000N R19L/P2000N R01L/P2000N -SN FEW006 SCT010 OVC027 M02/M03 Q1003=,MANUAL
2021-01-01 02:20:00,ENGM 010220Z 36006KT 6000 -SN FEW008 SCT026 OVC031 M02/M03 Q1003=,MANUAL
...,...,...
2023-12-31 21:50:00,ENGM 312150Z 04011KT 9000 -SN OVC008 M05/M07 Q1011 TEMPO 4000 -SN=,MANUAL
2023-12-31 22:20:00,ENGM 312220Z 04010KT 9999 OVC008 M05/M07 Q1011=,MANUAL
2023-12-31 22:50:00,ENGM 312250Z 04010KT 9999 -SN OVC009 M05/M07 Q1011=,MANUAL


In [110]:
timestamp_counts = metar_data.groupby(['airport_identifier', 'issued_at']).size().reset_index(name='timestamp_count')
metar_data = pd.merge(timestamp_counts, metar_data, on=['airport_identifier', 'issued_at'], how='left')

In [111]:
metar_data

,airport_identifier,issued_at,timestamp_count,metar,metartype
0,ENAL,2021-01-01 00:20:00,1,ENAL 010020Z 08005KT 9999 SCT024/// BKN037/// 01/M02 Q1006=,AUTO
1,ENAL,2021-01-01 00:50:00,1,ENAL 010050Z 09006KT 9999 FEW025/// 00/M02 Q1006=,AUTO
2,ENAL,2021-01-01 01:20:00,1,ENAL 010120Z 08005KT 9999 FEW027/// SCT040/// M00/M03 Q1006=,AUTO
3,ENAL,2021-01-01 01:50:00,1,ENAL 010150Z 09006KT 9999 BKN025/// M01/M02 Q1006=,AUTO
4,ENAL,2021-01-01 02:20:00,1,ENAL 010220Z 09007KT 9999 OVC026/// 01/M01 Q1006=,AUTO
...,...,...,...,...,...
3275708,ENZV,2023-12-31 21:50:00,1,ENZV 312150Z 10022KT CAVOK 05/M05 Q0994=,MANUAL
3275709,ENZV,2023-12-31 22:20:00,1,ENZV 312220Z 10023G34KT CAVOK 05/M05 Q0994=,MANUAL
3275710,ENZV,2023-12-31 22:50:00,1,ENZV 312250Z 10021KT CAVOK 05/M04 Q0994=,MANUAL
3275711,ENZV,2023-12-31 23:20:00,1,ENZV 312320Z 10024KT CAVOK 04/M05 Q0994=,MANUAL


In [103]:
metar_data[(metar_data['timestamp_count'] == 1) & (metar_data['airport_identifier'] == 'ENGM')]

,airport_identifier,issued_at,timestamp_count,metar,metartype
836288,ENGM,2021-01-01 00:20:00,1,ENGM 010020Z 36006KT 9999 4900E -SN FEW009 OVC017 M02/M03 Q1003=,MANUAL
836289,ENGM,2021-01-01 00:50:00,1,ENGM 010050Z 36003KT 9000 -SN FEW007 OVC018 M02/M03 Q1003=,MANUAL
836290,ENGM,2021-01-01 01:20:00,1,ENGM 010120Z 36006KT 8000 -SN FEW008 OVC029 M02/M03 Q1003=,MANUAL
836291,ENGM,2021-01-01 01:50:00,1,ENGM 010150Z 36006KT 9999 6000 R19R/P2000N R01R/P2000N R19L/P2000N R01L/P2000N -SN FEW006 SCT010 OVC027 M02/M03 Q1003=,MANUAL
836292,ENGM,2021-01-01 02:20:00,1,ENGM 010220Z 36006KT 6000 -SN FEW008 SCT026 OVC031 M02/M03 Q1003=,MANUAL
...,...,...,...,...,...
889390,ENGM,2023-12-31 21:50:00,1,ENGM 312150Z 04011KT 9000 -SN OVC008 M05/M07 Q1011 TEMPO 4000 -SN=,MANUAL
889391,ENGM,2023-12-31 22:20:00,1,ENGM 312220Z 04010KT 9999 OVC008 M05/M07 Q1011=,MANUAL
889392,ENGM,2023-12-31 22:50:00,1,ENGM 312250Z 04010KT 9999 -SN OVC009 M05/M07 Q1011=,MANUAL
889393,ENGM,2023-12-31 23:20:00,1,ENGM 312320Z 04010KT 9999 -SN OVC009 M05/M07 Q1011=,MANUAL


In [70]:
metar_FZ = metar_data[metar_data['metar'].str.contains('FZRA')]

In [72]:
metar_FZ.head(50)

,airport_identifier,issued_at,timestamp_count,metar,metartype
493,ENAL,2021-01-11 03:20:00,1,ENAL 110320Z 17007KT 9999 -FZRA FEW007/// BKN018/// OVC027/// M01/M02 Q1000 RERASN=,AUTO
496,ENAL,2021-01-11 04:50:00,2,ENAL 110450Z 20010KT 9999 -FZRA FEW003/// SCT008/// BKN012/// M01/M02 Q0999=,AUTO
1315,ENAL,2021-01-27 22:20:00,1,ENAL 272220Z 14012KT 9999 FEW008/// BKN011/// OVC014/// M01/M01 Q1007 REFZUP RESHUP REFZRA=,AUTO
1316,ENAL,2021-01-27 22:50:00,1,ENAL 272250Z 14010KT 2300 -SN FEW002/// OVC010/// M00/M01 Q1007 REFZUP RESHUP REFZRA=,AUTO
1322,ENAL,2021-01-28 01:50:00,1,ENAL 280150Z 13012KT 9999 -FZDZ SCT008/// OVC016/// M00/M01 Q1006 REFZUP RESHUP REFZRA=,AUTO
1324,ENAL,2021-01-28 02:50:00,1,ENAL 280250Z 15011KT 9999 -FZDZ FEW005/// BKN020/// M00/M02 Q1005 REFZUP RESHUP REFZRA=,AUTO
1466,ENAL,2021-01-31 00:50:00,1,ENAL 310050Z 34009KT 300V010 2200 -PL FEW005/// SCT014/// BKN020/// 01/M01 Q0999 REUP RESHUP REFZRA=,AUTO
1956,ENAL,2021-02-10 01:50:00,1,ENAL 100150Z 15010KT 9999 VV016 M03/M05 Q1020 REFZUP RESHUP REFZRA=,AUTO
1961,ENAL,2021-02-10 04:20:00,1,ENAL 100420Z 12010KT 5000 FZUP OVC014/// M02/M04 Q1021 REFZUP RESHUP REFZRA=,AUTO
3068,ENAL,2021-03-04 22:50:00,1,ENAL 042250Z 31009KT 280V360 6000 -SN FEW006/// SCT013/// BKN038/// 00/M01 Q1026 RESHUP REFZRA=,AUTO
